<a href="https://colab.research.google.com/github/avinashronanki/hyperparameter_tuning/blob/master/hyperparameter.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#https://towardsdatascience.com/model-hyperparameters-tuning-using-grid-random-and-genetic-based-search-in-python-4807b580ee01

In [1]:
!pip install tpot

     |████████████████████████████████| 92kB 2.5MB/s 
     |████████████████████████████████| 163kB 8.6MB/s 
  Created wheel for stopit: filename=stopit-1.1.2-cp36-none-any.whl size=11956 sha256=315427babc4746dde3b48d33cd304f4abd88fe17a2cc5b9f48a0773a5709ceb4
  Stored in directory: /root/.cache/pip/wheels/3c/85/2b/2580190404636bfc63e8de3dff629c03bb795021e1983a6cc7
Successfully built stopit


In [2]:
import pandas as pd    # Data manipulation
import numpy as np     # Array manipulation
# GBM regressor
from sklearn.ensemble import GradientBoostingRegressor
# Train Test Split
from sklearn.model_selection import train_test_split
# Data Preprocessing (Scaling)
from sklearn.preprocessing import StandardScaler
# Model Evaluation
from sklearn.metrics import mean_absolute_error
# Grid and Random CV Search
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
# Genetic Algorithm based Search
from tpot import TPOTRegressor

In [3]:
concrete = pd.read_excel("/content/Concrete.xlsx")
concrete.head()

,Cement,BF_slag,Fly_ash,Water,Super_plasticizer,Coarse_aggregate,Fine_aggregate,Age,Comp_str
0,540.0,0.0,0.0,162.0,2.5,1040.0,676.0,28,79.986111
1,540.0,0.0,0.0,162.0,2.5,1055.0,676.0,28,61.887366
2,332.5,142.5,0.0,228.0,0.0,932.0,594.0,270,40.269535
3,332.5,142.5,0.0,228.0,0.0,932.0,594.0,365,41.052780
4,198.6,132.4,0.0,192.0,0.0,978.4,825.5,360,44.296075


In [4]:
concrete.columns # view columns 

Index(['Cement', 'BF_slag', 'Fly_ash', 'Water', 'Super_plasticizer',
       'Coarse_aggregate', 'Fine_aggregate', 'Age', 'Comp_str'],
      dtype='object')

In [5]:
concrete.info() # view the info 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1030 entries, 0 to 1029
Data columns (total 9 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Cement             1030 non-null   float64
 1   BF_slag            1030 non-null   float64
 2   Fly_ash            1030 non-null   float64
 3   Water              1030 non-null   float64
 4   Super_plasticizer  1030 non-null   float64
 5   Coarse_aggregate   1030 non-null   float64
 6   Fine_aggregate     1030 non-null   float64
 7   Age                1030 non-null   int64  
 8   Comp_str           1030 non-null   float64
dtypes: float64(8), int64(1)
memory usage: 72.5 KB


In [6]:
concrete.describe() #basic descriptive statistics

,Cement,BF_slag,Fly_ash,Water,Super_plasticizer,Coarse_aggregate,Fine_aggregate,Age,Comp_str
count,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000
mean,281.165631,73.895485,54.187136,181.566359,6.203112,972.918592,773.578883,45.662136,35.817836
std,104.507142,86.279104,63.996469,21.355567,5.973492,77.753818,80.175427,63.169912,16.705679
min,102.000000,0.000000,0.000000,121.750000,0.000000,801.000000,594.000000,1.000000,2.331808
25%,192.375000,0.000000,0.000000,164.900000,0.000000,932.000000,730.950000,7.000000,23.707115
50%,272.900000,22.000000,0.000000,185.000000,6.350000,968.000000,779.510000,28.000000,34.442774
75%,350.000000,142.950000,118.270000,192.000000,10.160000,1029.400000,824.000000,56.000000,46.136287
max,540.000000,359.400000,200.100000,247.000000,32.200000,1145.000000,992.600000,365.000000,82.599225


In [7]:
X = concrete.drop("Comp_str", axis = 1)
y = concrete["Comp_str"]
print(X.shape)
print(y.shape)

(1030, 8)
(1030,)


In [9]:
X.head()

,Cement,BF_slag,Fly_ash,Water,Super_plasticizer,Coarse_aggregate,Fine_aggregate,Age
0,540.0,0.0,0.0,162.0,2.5,1040.0,676.0,28
1,540.0,0.0,0.0,162.0,2.5,1055.0,676.0,28
2,332.5,142.5,0.0,228.0,0.0,932.0,594.0,270
3,332.5,142.5,0.0,228.0,0.0,932.0,594.0,365
4,198.6,132.4,0.0,192.0,0.0,978.4,825.5,360


In [10]:
y.head()

0    79.986111
1    61.887366
2    40.269535
3    41.052780
4    44.296075
Name: Comp_str, dtype: float64

In [11]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.20, random_state = 42) #spliting the data 

In [12]:
# scale the variables so that the algorithms optimize the learning process faster
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [13]:
clf = GradientBoostingRegressor(criterion = "mae" ) # type of parameters
clf.get_params()

{'alpha': 0.9,
 'ccp_alpha': 0.0,
 'criterion': 'mae',
 'init': None,
 'learning_rate': 0.1,
 'loss': 'ls',
 'max_depth': 3,
 'max_features': None,
 'max_leaf_nodes': None,
 'min_impurity_decrease': 0.0,
 'min_impurity_split': None,
 'min_samples_leaf': 1,
 'min_samples_split': 2,
 'min_weight_fraction_leaf': 0.0,
 'n_estimators': 100,
 'n_iter_no_change': None,
 'presort': 'deprecated',
 'random_state': None,
 'subsample': 1.0,
 'tol': 0.0001,
 'validation_fraction': 0.1,
 'verbose': 0,
 'warm_start': False}

In [14]:
# Create a Gradient Boosted Regressor with specified criterion
gb_regressor = GradientBoostingRegressor(criterion = "mae")
# Create the parameter grid
param_grid = {'max_depth' : [2, 4, 8, 10, 12],
              'n_estimators' : [100, 200, 300],
              'max_features' : ['auto', 'sqrt'],
              "criterion" : ["friedman_mse", "mse", "mae"]}
# Create a GridSearchCV object
grid_gb = GridSearchCV(
    estimator = gb_regressor,
    param_grid = param_grid,
    scoring = 'neg_mean_absolute_error',
    n_jobs = 4,
    cv = 10,
    refit = True,
    return_train_score = True)
print(grid_gb)

GridSearchCV(cv=10, error_score=nan,
             estimator=GradientBoostingRegressor(alpha=0.9, ccp_alpha=0.0,
                                                 criterion='mae', init=None,
                                                 learning_rate=0.1, loss='ls',
                                                 max_depth=3, max_features=None,
                                                 max_leaf_nodes=None,
                                                 min_impurity_decrease=0.0,
                                                 min_impurity_split=None,
                                                 min_samples_leaf=1,
                                                 min_samples_split=2,
                                                 min_weight_fraction_leaf=0.0,
                                                 n_estimators=100,
                                                 n_iter_no_change...
                                                 random_state=None,
         

In [15]:
grid_gb.fit(X_train_scaled, y_train)

GridSearchCV(cv=10, error_score=nan,
             estimator=GradientBoostingRegressor(alpha=0.9, ccp_alpha=0.0,
                                                 criterion='mae', init=None,
                                                 learning_rate=0.1, loss='ls',
                                                 max_depth=3, max_features=None,
                                                 max_leaf_nodes=None,
                                                 min_impurity_decrease=0.0,
                                                 min_impurity_split=None,
                                                 min_samples_leaf=1,
                                                 min_samples_split=2,
                                                 min_weight_fraction_leaf=0.0,
                                                 n_estimators=100,
                                                 n_iter_no_change...
                                                 random_state=None,
         

In [16]:
grid_gb.best_score_

-2.92504271618051

In [17]:
cv_results = pd.DataFrame(grid_gb.cv_results_)

In [ ]:
cv_results

In [19]:
# Extract and print the row that had the best mean test score
best_row = cv_results[cv_results['rank_test_score'] == 1]
print(best_row)

    mean_fit_time  std_fit_time  ...  mean_train_score  std_train_score
11       0.530222      0.005021  ...         -1.172779         0.043803

[1 rows x 34 columns]


In [20]:
grid_gb.best_params_["n_estimators"]

300

In [21]:
predictions = grid_gb.best_estimator_.predict(X_test_scaled)
mean_absolute_error(predictions, y_test)

2.7586308471560006

In [ ]:
# Random Grid Search
